<a href="https://colab.research.google.com/github/pushkar-agni04/ML-with-GPU/blob/main/ML_with_GPU_acceleration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Goal
Build a high-performance machine learning pipeline to predict a target variable (e.g., housing prices, ride-share fares, or a similar regression/classification task with a large public dataset) and demonstrate a significant speedup versus the CPU baseline, culminating in a production-ready, optimized model

# Installing required libraries

In [1]:
# Check for GPU and CUDA version
!nvidia-smi

Sun Nov  9 16:20:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Installing xgboost version that supports cuda parameters for GPU acceleration
!apt install -y libnccl2
!pip install --upgrade xgboost==1.7.5 --extra-index-url=https://pypi.nvidia.com

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Some packages could not be installed. This may mean that you have
requested an impossible situation or if you are using the unstable
distribution that some required packages have not yet been created
or been moved out of Incoming.
The following information may help to resolve the situation:

The following packages have unmet dependencies:
 libnccl-dev : Depends: libnccl2 (= 2.22.3-1+cuda12.5) but 2.28.7-1+cuda13.0 is to be installed
E: Error, pkgProblemResolver::Resolve generated breaks, this may be caused by held packages.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 6.1 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 3.1.1
    Uninstalling xgboost-3.1.1:
      Successfully uninstalled xgboost-3.1.1


In [3]:
import sys, os, time

In [4]:
import cudf
import cuml
import pandas as pd
import numpy as np
import xgboost as xgb
from cuml.model_selection import train_test_split

In [5]:
print(f"cuDF version: {cudf.__version__}")
print(f"cuML version: {cuml.__version__}")

cuDF version: 25.06.00
cuML version: 25.06.00


# 1. Accelerated data loading


In [6]:
# Downloading a medium-sized dataset for a meaningful speed test

print("Downloading NYC Yellow Taxi Parquet file (Jan 2024)...")
!wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
!mv 'yellow_tripdata_2024-01.parquet' 'taxi_data.parquet'

--2025-11-09 16:20:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.97.13, 52.85.97.61, 52.85.97.194, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.97.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49961641 (48M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-01.parquet’

yellow_tripdata_202 100%[===================>]  47.65M   221MB/s    in 0.2s    

2025-11-09 16:20:45 (221 MB/s) - ‘yellow_tripdata_2024-01.parquet’ saved [49961641/49961641]



In [7]:
# Verify the file is downloaded
!ls -lh taxi_data.parquet

-rw-r--r-- 1 root root 48M Mar 21  2024 taxi_data.parquet


In [8]:
# Define the columns we are interested in for our regression task
TARGET_COLS = ['passenger_count', 'trip_distance', 'fare_amount', 'tip_amount', 'total_amount']

**Loading data to CPU Pandas for benchmarking**

In [9]:
# --- Load to CPU Pandas DataFrame for benchmarking ---
print("\nLoading data onto CPU (Pandas)...")
start_time_cpu = time.time()
cpu_df = pd.read_parquet('taxi_data.parquet', columns=TARGET_COLS)
cpu_load_time = time.time() - start_time_cpu
print(f"CPU DataFrame shape: {cpu_df.shape}")
print(f"Time to load CPU (Pandas): {cpu_load_time:.4f} seconds")


Loading data onto CPU (Pandas)...
CPU DataFrame shape: (2964624, 5)
Time to load CPU (Pandas): 0.3029 seconds


**Loading data to GPU Cudf for benchmarking**

In [11]:
# --- Load to GPU cuDF DataFrame for benchmarking ---
print("\nLoading data onto GPU (cuDF)...")
start_time_gpu = time.time()
gpu_df = cudf.read_parquet('taxi_data.parquet', columns=TARGET_COLS)
gpu_load_time = time.time() - start_time_gpu
print(f"GPU DataFrame shape: {gpu_df.shape}")
print(f"Time to load GPU (cuDF): {gpu_load_time:.4f} seconds")


Loading data onto GPU (cuDF)...
GPU DataFrame shape: (2964624, 5)
Time to load GPU (cuDF): 0.1154 seconds


In [12]:
# --- Comparison ---
speedup_load = cpu_load_time / gpu_load_time
print(f"\n--- Load Performance Summary ---")
print(f"🔥 GPU Speedup (Load Time): {speedup_load:.2f}x faster!")


--- Load Performance Summary ---
🔥 GPU Speedup (Load Time): 2.62x faster!


# 2. Accelerated Feature Engineering

In [13]:
print("\n--- Benchmarking CPU (Pandas) GroupBy ---")
start_cpu = time.time()
# Note: We filter for valid data points to prevent errors
cpu_result = cpu_df[cpu_df['passenger_count'] > 0].groupby('passenger_count')['fare_amount'].mean().reset_index()
cpu_elapsed = time.time() - start_cpu
print(f"CPU result (first 5 rows):\n{cpu_result.head()}")
print(f"Time for CPU GroupBy: {cpu_elapsed:.6f} seconds")


--- Benchmarking CPU (Pandas) GroupBy ---
CPU result (first 5 rows):
   passenger_count  fare_amount
0              1.0    17.557052
1              2.0    20.171285
2              3.0    20.041299
3              4.0    21.833799
4              5.0    17.511870
Time for CPU GroupBy: 0.169564 seconds


In [15]:
# 2. --- cuDF (GPU) GroupBy Benchmark ---
print("\n--- Benchmarking GPU (cuDF) GroupBy ---")
start_gpu = time.time()
gpu_result = gpu_df[gpu_df['passenger_count'] > 0].groupby('passenger_count')['fare_amount'].mean().reset_index()
gpu_elapsed = time.time() - start_gpu
print(f"GPU result (first 5 rows):\n{gpu_result.head().to_pandas()}")
print(f"Time for GPU GroupBy: {gpu_elapsed:.6f} seconds")


--- Benchmarking GPU (cuDF) GroupBy ---
GPU result (first 5 rows):
   passenger_count  fare_amount
0                9    11.400000
1                2    20.171285
2                4    21.833799
3                7    45.411250
4                1    17.557052
Time for GPU GroupBy: 0.018535 seconds


In [16]:
# 3. --- Comparison ---
speedup = cpu_elapsed / gpu_elapsed
print(f"\n--- Performance Summary (GroupBy) ---")
print(f"CPU time: {cpu_elapsed:.4f}s")
print(f"GPU time: {gpu_elapsed:.4f}s")
print(f"🔥 GPU Speedup: {speedup:.2f}x faster!")


--- Performance Summary (GroupBy) ---
CPU time: 0.1696s
GPU time: 0.0185s
🔥 GPU Speedup: 9.15x faster!


# 3. Accelerated model training

In [17]:
import xgboost as xgb
from cuml.model_selection import train_test_split

# --- 1. Prepare Data ---
FEATURE_COLS = ['passenger_count', 'trip_distance', 'tip_amount', 'total_amount']
TARGET_COL = 'fare_amount'

# Ensure data is clean (e.g., non-negative features, valid total amount)
gpu_clean_df = gpu_df[(gpu_df[TARGET_COL] > 0) & (gpu_df['trip_distance'] > 0) & (gpu_df['total_amount'] > 0) & (gpu_df['passenger_count'] > 0)]

# Convert cuDF DataFrame columns to CuPy arrays (XGBoost optimized format)
X_gpu = gpu_clean_df[FEATURE_COLS].values
y_gpu = gpu_clean_df[TARGET_COL].values

# Split data using cuML's train_test_split (performs splitting directly on the GPU)
X_train, X_test, y_train, y_test = train_test_split(X_gpu, y_gpu, test_size=0.2, random_state=42)

In [18]:
# --- 2. Train CPU Model (Standard XGBoost) ---
print("\n--- Training CPU XGBoost (Reference) ---")

# Transfer data to CPU for CPU-only DMatrix
dtrain_cpu = xgb.DMatrix(X_train.get(), y_train.get())
cpu_params = {'objective': 'reg:squarederror', 'tree_method': 'hist', 'random_state': 42}

start_cpu_train = time.time()
bst_cpu = xgb.train(cpu_params, dtrain_cpu, num_boost_round=100) # 100 rounds is a good benchmark
cpu_train_time = time.time() - start_cpu_train
print(f"CPU Training Time: {cpu_train_time:.4f} seconds")


--- Training CPU XGBoost (Reference) ---
CPU Training Time: 7.8594 seconds


In [19]:
# --- 3. Train GPU Model (XGBoost-CUDA) ---
print("\n--- Training GPU XGBoost-CUDA (Accelerated) ---")
# Use the GPU data (no .get() needed)
dtrain_gpu = xgb.DMatrix(X_train, y_train)

# KEY PARAMETER: 'gpu_hist' enables GPU acceleration
gpu_params = {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'random_state': 42}

start_gpu_train = time.time()
bst_gpu = xgb.train(gpu_params, dtrain_gpu, num_boost_round=100)
gpu_train_time = time.time() - start_gpu_train
print(f"GPU Training Time: {gpu_train_time:.4f} seconds")



--- Training GPU XGBoost-CUDA (Accelerated) ---
GPU Training Time: 0.7633 seconds


In [20]:
# --- 4. Comparison ---
speedup_train = cpu_train_time / gpu_train_time
print(f"\n--- Training Performance Summary ---")
print(f"CPU time: {cpu_train_time:.4f}s")
print(f"GPU time: {gpu_train_time:.4f}s")
print(f"🚀 GPU Training Speedup: {speedup_train:.2f}x faster!")

# Save the GPU model for Phase 2 optimization
bst_gpu.save_model("gpu_trained_model.xgb")
print("\nGPU-trained XGBoost model saved as 'gpu_trained_model.xgb'")


--- Training Performance Summary ---
CPU time: 7.8594s
GPU time: 0.7633s
🚀 GPU Training Speedup: 10.30x faster!

GPU-trained XGBoost model saved as 'gpu_trained_model.xgb'


# Checking the effect of data size (10x) with GPU accelleration
- GPU acceleration is significantly more with larger sized datasets

In [23]:
vintages = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10']

# Define the base URL structure
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"

for vintage in vintages:
    download_url = f"{BASE_URL}{vintage}.parquet"
    output_filename = f"taxi_data_{vintage}.parquet"
    source_filename = f"yellow_tripdata_{vintage}.parquet"

    print(f"\n--- Downloading NYC Yellow Taxi Parquet file for {vintage} ---")
    !wget -nc {download_url} -O {output_filename}

    # Optional: Verify the file exists
    if os.path.exists(output_filename):
        print(f"✅ Successfully downloaded and saved as {output_filename}")
    else:
        print(f"❌ Error downloading {vintage}")


--- Downloading NYC Yellow Taxi Parquet file for 2024-01 ---
--2025-11-09 16:23:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.97.13, 52.85.97.61, 52.85.97.194, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.97.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49961641 (48M) [binary/octet-stream]
Saving to: ‘taxi_data_2024-01.parquet’

taxi_data_2024-01.p 100%[===================>]  47.65M   257MB/s    in 0.2s    

2025-11-09 16:23:41 (257 MB/s) - ‘taxi_data_2024-01.parquet’ saved [49961641/49961641]

✅ Successfully downloaded and saved as taxi_data_2024-01.parquet

--- Downloading NYC Yellow Taxi Parquet file for 2024-02 ---
--2025-11-09 16:23:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)...

In [24]:
import glob

# 1. Setup
FEATURE_COLS = ['passenger_count', 'trip_distance', 'tip_amount', 'total_amount']
TARGET_COL = 'fare_amount'

# Create a list of all downloaded file paths
file_list = glob.glob('taxi_data_2024-*.parquet')

if not file_list:
    print("\n🚨 ERROR: No parquet files found. Please run Step 1.")
else:
    print(f"\n--- Found {len(file_list)} files. Starting GPU consolidation... ---")

    # --- GPU Consolidation ---
    start_time_gpu_read = time.time()
    # Read all files directly into one cuDF DataFrame
    gpu_df = cudf.read_parquet(file_list, columns=FEATURE_COLS + [TARGET_COL])
    gpu_read_time = time.time() - start_time_gpu_read

    # --- Clean Data (GPU side) ---
    gpu_df = gpu_df[(gpu_df[TARGET_COL] > 0) & (gpu_df['trip_distance'] > 0) & (gpu_df['total_amount'] > 0) & (gpu_df['passenger_count'] > 0)]


    print(f"Total consolidated rows (GPU): {len(gpu_df):,}")
    print(f"Time to read and clean GPU data: {gpu_read_time:.2f} seconds.")

    # --- CPU Copy (Explicitly timing the GPU -> CPU transfer) ---
    print("\n--- Transferring GPU data to CPU (Pandas) for Baseline ---")
    start_time_transfer = time.time()
    # .to_pandas() copies the data from VRAM to CPU RAM
    cpu_df = gpu_df.to_pandas()
    transfer_time = time.time() - start_time_transfer

    print(f"Total rows (CPU): {len(cpu_df):,}")
    print(f"Time for GPU -> CPU transfer: {transfer_time:.2f} seconds.")

    # Basic check to ensure the transfer was successful
    if len(gpu_df) != len(cpu_df):
        print("Mismatched row count after transfer!")


--- Found 10 files. Starting GPU consolidation... ---
Total consolidated rows (GPU): 29,281,113
Time to read and clean GPU data: 0.23 seconds.

--- Transferring GPU data to CPU (Pandas) for Baseline ---
Total rows (CPU): 29,281,113
Time for GPU -> CPU transfer: 1.89 seconds.


In [25]:
# 1. Feature/Target Setup (using global names from Step 2)
FEATURE_COLS = ['passenger_count', 'trip_distance', 'tip_amount', 'total_amount']
TARGET_COL = 'fare_amount'

# --- Part A: GroupBy Aggregation Benchmark ---
print("\n================================================")
print("  PART A: GroupBy Aggregation Benchmark")
print("================================================")

# Pandas (CPU) GroupBy Benchmark
start_cpu = time.time()
cpu_df.groupby('passenger_count')['fare_amount'].mean()
cpu_elapsed = time.time() - start_cpu
print(f"Time for CPU GroupBy: {cpu_elapsed:.4f} seconds")

# cuDF (GPU) GroupBy Benchmark
start_gpu = time.time()
gpu_df.groupby('passenger_count')['fare_amount'].mean()
gpu_elapsed = time.time() - start_gpu
print(f"Time for GPU GroupBy: {gpu_elapsed:.4f} seconds")

# Comparison
speedup = cpu_elapsed / gpu_elapsed
print(f"🔥 GPU GroupBy Speedup: {speedup:.2f}x faster!")


  PART A: GroupBy Aggregation Benchmark
Time for CPU GroupBy: 2.3556 seconds
Time for GPU GroupBy: 0.0431 seconds
🔥 GPU GroupBy Speedup: 54.65x faster!


In [26]:
# --- Part B: XGBoost Training Benchmark ---
print("\n================================================")
print("  PART B: XGBoost Training Benchmark")
print("================================================")

# Prepare Data (GPU side first)
X_gpu = gpu_df[FEATURE_COLS].values
y_gpu = gpu_df[TARGET_COL].values
X_train, X_test, y_train, y_test = train_test_split(X_gpu, y_gpu, test_size=0.2, random_state=42)


# Train CPU Model
print("Training CPU XGBoost (Reference)...")
dtrain_cpu = xgb.DMatrix(X_train.get(), y_train.get())
cpu_params = {'objective': 'reg:squarederror', 'tree_method': 'hist', 'random_state': 42}
start_cpu_train = time.time()
bst_cpu = xgb.train(cpu_params, dtrain_cpu, num_boost_round=100)
cpu_train_time = time.time() - start_cpu_train
print(f"CPU Training Time: {cpu_train_time:.4f} seconds")

# Train GPU Model
print("Training GPU XGBoost-CUDA (Accelerated)...")
dtrain_gpu = xgb.DMatrix(X_train, y_train)
# Ensure gpu_hist is available from the fixes we ran previously
gpu_params = {'objective': 'reg:squarederror', 'tree_method': 'gpu_hist', 'random_state': 42}

start_gpu_train = time.time()
bst_gpu = xgb.train(gpu_params, dtrain_gpu, num_boost_round=100)
gpu_train_time = time.time() - start_gpu_train
print(f"GPU Training Time: {gpu_train_time:.4f} seconds")

# Comparison
speedup_train = cpu_train_time / gpu_train_time
print(f"🚀 GPU Training Speedup: {speedup_train:.2f}x faster!")


  PART B: XGBoost Training Benchmark
Training CPU XGBoost (Reference)...
CPU Training Time: 118.6900 seconds
Training GPU XGBoost-CUDA (Accelerated)...
GPU Training Time: 6.0251 seconds
🚀 GPU Training Speedup: 19.70x faster!
